## Using OperationDefinition to generate Observation instances

See discussion on [Zulip](https://chat.fhir.org/#narrow/stream/179166-implementers/topic/Generic.20or.20specific.20profiles.3F) regarding the common and simple need for a way to use [ObservationDefinition](http://build.fhir.org/observationdefinition.html) as table or spreadsheet to create Observation instances based on a few 'base' profiles.ObservationDefinition needs to be structured simply and flat enough to be able to directly transform into a table to contain the stuff you need for creating an Observation.

The following proof of concept Python script demonstrates this.  It takes the following input artifacts:

1. patients results
2. ObservationDefinition instances **represented by a row in a CSV file**
3. a common Observation template (i.e. a declared Observation Profile)

to generates Observation instances.

Each row of the ObservationDefinition table is applied to a nominated Observation profile to create an Observation "form".  The patient data is inserted into this "form" to create the FHIR Observation for that instance.

The set of data about the tests whether represented as a bundle or as a in a table is like a "Dictionary of Observation type" for use.  This is a shortcut to creating 100s or 1000s of individual profiles.  Instead you would need only a few common Observation Profile for each type ( e.g. quantitative ) of Observation.

This use case extends ObservationDefinition scope to include *both* for informing the contents of a Service Catalog or Instrument Specification as well as providing the constraints for the Observations.  This scope would overlap with StructureDefinition.

**Skip down to [Create Observations](#another_cell) to see this in action  the first part is a bunch of Python code to set it up.**

General Python Instructions:
Run in python 3.6

Outline:
    
1. Read rows from ObservationDefinition(OD) spreadsheet representation to namedTuple
2. Apply OD data to named profile (SD)
3. Write/display/validate profile

## Imports

In [ ]:
from datetime import datetime
from collections import namedtuple
from pandas import read_csv, DataFrame
from json import dumps
from requests import post, get
from IPython.display import display, Markdown

## Global Variables 

In [ ]:
# in_file = '/Users/ehaas/Documents/Python/Notebooks/OD - od.csv'
in_file = 'OD_to_Obs.csv'
sheet_name = 'od'
now = f'{datetime.utcnow().isoformat()}Z'
outfile = 'outfile'  # name of output file
headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}
fhir_test_server = 'http://test.fhir.org/r4'

## Fetch data from csv file

In [ ]:
# dataframes are a pain so will reload as a csv and
# use named tuple structure
def get_testinfo():
    with open(in_file, encoding = "ISO-8859-1") as f:
        reader = csv.reader(f)
        top_row = next(reader)
        # top_row = [t.lower().split(' ') for t in top_row]
        # top_row = ['_'.join(t) for t in top_row]
        # for t in top_row:
            # print(t)
        Data = namedtuple("Data", top_row)
        # next(reader) # skip 2nd row of definitions
        data = [Data(*r) for r in reader]
        return data

    # using named tuple structure make getting data all nice n pretty like
    # allow one to use dot notation like 'data[0].name'

    #print(f'name = {data[0].Name}\
    #      \ndescription = {data[0].Description}\
    #      \ncode = {data[0].codeCoding_0_Code}')
# d = get_testinfo()
# print(d[0].OD_id)


<a id='another_cell'></a>
## Create Observations

### Gather patient information (just a namedtuple for this demo instead of FHIR object)

In [ ]:
obs_list=[]
# get patient (just a namedtuple for this demo instead of FHIR object)
Patient = namedtuple('Patient','id name')
patient = Patient('1234','Jane Doe')

print(f'Patient Name = {patient.name},  Patient Id = {patient.id}')

### Preview ObservationDefinition Data about the test  from csv file

In [ ]:
df = read_csv(in_file, encoding = "ISO-8859-1")
# to convert to named tuple
#Data = namedtuple('Data', df.columns)
#data = [Data(*r[1:]) for r in df.itertuples() if r[0] > 0]
print('Test=ObservationDefinition data:')
df.dropna(axis='columns')

###  Get Test Result Data - (just a dictionary or hash for this example)

In [ ]:
# get test result data - (just a dictionary or hash for this example)
# k = test name : v = (result value, timestamp)
results = dict(
    bg = (96.0, now),
    bun = (20.0, now),
    na = (135.0, now)
    ) 
print('Test Result Values:')
DataFrame([[k] + list(v) for k,v in results.items()], columns=['Test Name', 'Value', "TimeStamp"])

### Observation template (profile instance)

This is a function that returns a fully populated template.

In [ ]:
def get_obs(t, p, r, s):# apply od spreadsheet data , patient info, results and status to profile 
    value, effective = r
    return {
      'resourceType': 'Observation',
      'meta': { 'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-observationresults'],
      },
      'status': s,
      'category': [{
        'coding': [
          {
            'system': 'http://hl7.org/fhir/observation-category',
            'code': 'laboratory',
            'display': 'Laboratory'
          }
        ],
        'text': 'Laboratory'
      }],
      'code': {
        'coding': [
          {
            'system': 'http://loinc.org',
            'code': t.OD_code_coding_0_code,
            'display': t.OD_code_coding_0_display
          }
        ],
        'text': t.OD_code_text
      },
      'subject': {
        'reference': f'Patient/{p.id}',
          'display': p.name
      },
        
      'effectiveDateTime': effective,
      'valueQuantity': {
        'value': value,
        'unit': t.OD_quantitativeDetails_customaryUnit_coding_0_code,
        'system': t.OD_quantitativeDetails_unit_coding_0_system,
        'code': t.OD_quantitativeDetails_unit_coding_0_code
          },
      'referenceRange': [
        {
          'low': {
            'value': float(t.OD_qualifiedInterval_0_range_low_value),
            'unit': t.OD_quantitativeDetails_customaryUnit_coding_0_code,
            'system': t.OD_quantitativeDetails_unit_coding_0_system,
            'code': t.OD_quantitativeDetails_unit_coding_0_code
          },
          'high': {
            'value': float(t.OD_qualifiedInterval_0_range_high_value),
            'unit': t.OD_quantitativeDetails_customaryUnit_coding_0_code,
            'system': t.OD_quantitativeDetails_unit_coding_0_system,
            'code': t.OD_quantitativeDetails_unit_coding_0_code
          },
          'type': {
            'coding': [
              {
                'system': 'http://hl7.org/fhir/referencerange-meaning',
                'code': t.OD_qualifiedInterval_0_context_coding_0_code,
              }
            ]
           }
         }
       ]     
     }




### Create Observation Instance with above data as inputs 

In [ ]:
print('get test information from od spreadsheets:\n\n')
tests = get_testinfo()

for t in tests:
    new_obs = get_obs(t=t, p=patient, r=results[t.OD_id], s='preliminary' )
    print('='*79,'\n','='*30,f'{patient.name} {t.OD_name} results','='*30,'\n','='*79)
    print(f'Observation = {dumps(new_obs,indent = 3)}')
    obs_list.append(new_obs)


##  Add Narrative By POSTing and GETting a Resource from the Test Server
By POSTING it to the FHIR reference implementation server, the example is validated and text narrative generated.  (you can create your own narrative using Jinja2 for example but this is an easy hack)  The results are displayed below in the human readable text as xhtml.


In [ ]:
# validate using requests
rp = post(f'{fhir_test_server}/Observation',headers = headers, data = dumps(obs_list[0]))
print(rp.status_code)
print(rp.json()['id'])
rg = get(f'{fhir_test_server}/Observation/{rp.json()["id"]}',headers = headers )
# view  output
display(Markdown(f'<h1>Output</h1>{rg.json()["text"]["div"]}'))
display(Markdown('### Json:'))
display(Markdown(dumps(rg.json(), indent=3)))


## FIN